In [2]:
import numpy as np
import cv2

BLUE, GREEN, RED, BLACK, WHITE = (255,0,0), (0,255,0), (0,0,255), (0,0,0), (255,255,255)
DRAW_BG = {'color': BLACK, 'val': 0}
DRAW_FG = {'color': WHITE, 'val': 1}

rect = (0,0,1,1)
drawing = False
rectangle = False
rect_over = False
rect_or_mask = 100
value = DRAW_FG
thickness = 3

def onMouse(event, x, y, flags, param):
    global ix, iy, img, img2, drawing, value, mask, rectangle
    global rect, rect_or_mask, rect_over
    
    if event == cv2.EVENT_RBUTTONDOWN:
        rectangle = True
        ix, iy = x, y
    
    elif event == cv2.EVENT_MOUSEMOVE:
        if rectangle:
            img = img2.copy()
            cv2.rectangle(img, (ix,iy), (x,y), RED, 2)
            rect = (min(ix,x), min(iy,y), abs(ix-x), abs(iy-y))
            rect_or_mask = 0
    
    elif event == cv2.EVENT_RBUTTONUP:
        rectangle = False
        rect_over = True
        
        cv2.rectangle(img, (ix,iy,), (x,y), RED, 2)
        rect = (min(ix,x), min(iy,y), abs(ix-x), abs(iy-y))
        rect_or_mask = 0
        
    
    if event == cv2.EVENT_LBUTTONDOWN:
        if not rect_over:
            print('Draw a rectangle first using the right mouse button')
        
        else:
            drawing = True
            cv2.circle(img, (x,y), thickness, value['color'], -1)
            cv2.circle(mask, (x,y), thickness, value['val'], -1)
        
    elif event == cv2.EVENT_MOUSEMOVE:
        if drawing:
            cv2.circle(img, (x,y), thickness, value['color'], -1)
            cv2.circle(mask, (x,y), thickness, value['val'], -1)
            
    elif event == cv2.EVENT_LBUTTONUP:
        if drawing:
            drawing = False
            cv2.circle(img, (x,y), thickness, value['color'], -1)
            cv2.circle(mask, (x,y), thickness, value['val'], -1)
        
    

def grabcut():
    global ix, iy, img, img2, drawing, value, mask, rectangle
    global rect, rect_or_mask, rect_over
    
    img = cv2.imread('foot.jpg')
    img = cv2.resize(img, None, fx=0.5, fy=0.5)
    img2 = img.copy()
    
    mask = np.zeros(img.shape[:2], dtype=np.uint8) # 이미지와 똑같은?? 만듦?
    output = np.zeros(img.shape, np.uint8)
    
    cv2.namedWindow('input')
    cv2.namedWindow('output')
    cv2.setMouseCallback('input', onMouse, param=(img, img2))

    while True:
        cv2.imshow('input', img)
        cv2.imshow('output', output)
        
        k = cv2.waitKey(1) & 0xff
        
        if k == 27:
            break
            
        if k == ord('0'):
            print('>> for foreground')
            value = DRAW_BG
        elif k == ord('1'):
            print('>> for background')
            value = DRAW_FG
        elif k == ord('r'):
            print('>> reset')
            rect = (0,0,1,1)
            drawing = False
            rectangle = False
            rect_or_mask = 100
            rect_over = False
            value = DRAW_FG
            img = img2.copy()
            mask = np.zeros(img.shape[:2], dtype=np.uint8)
            output = np.zeros_like(img, dtype=np.uint8)
        elif k == ord('n'):
            bgdModel = np.zeros((1,65), np.float64)
            fgdModel = np.zeros((1,65), np.float64)
            
            if rect_or_mask == 0:
                cv2.grabCut(img2, mask, rect, bgdModel, fgdModel, 1, cv2.GC_INIT_WITH_RECT)
                rect_or_mask = 1
            elif rect_or_mask == 1:
                cv2.grabCut(img2, mask, rect, bgdModel, fgdModel, 1, cv2.GC_INIT_WITH_MASK)
                
        mask2 = np.where((mask==2)|(mask==0),0,1).astype('uint8')
        output = img2*mask2[:,:,np.newaxis]

         ## mask 확인용
        tmp = np.full_like(mask, 128)
        tmp[mask==0] = 0
        tmp[mask==1] = 255
        cv2.imshow('mask', tmp)

    print(np.unique(mask))
    
    cv2.destroyAllWindows()

In [2]:
grabcut()

[0]


In [3]:
import numpy as np

img = cv2.imread('butterfly.png')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

## SIFT 검출기 생성
sift = cv2.xfeatures2d.SIFT_create()

## SIFT 특징 검출
keypoints = sift.detect(image=gray, 
                        mask=None)

## keypoint 시각화
## https://docs.opencv.org/2.4/modules/features2d/doc/drawing_function_of_keypoints_and_matches.html#drawkeypoints
res = cv2.drawKeypoints(image = gray, 
                        keypoints = keypoints,
                        outImage = None)

res_with_rich = cv2.drawKeypoints(image = gray, 
                                  keypoints = keypoints,
                                  outImage = None,
                                  flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

cv2.imshow('keypoints', res)
cv2.imshow('keypoints_with_size_and_orientation', res_with_rich)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
import numpy as np
import cv2

# 이미지 불러오기
img_query = cv2.imread('box.png')           # query image

## 이미지 회전, 크기 변환 적용
height, width = img_query.shape[:2]
M = cv2.getRotationMatrix2D(center = (width/2, height/2),
                            angle = 45,
                            scale = 1.5)
img_query = cv2.warpAffine(src = img_query, 
                           M = M,
                           dsize = (width, height))

img_train = cv2.imread('box_in_scene.png')  # train image
gray_query = cv2.cvtColor(img_query, cv2.COLOR_BGR2GRAY)
gray_train = cv2.cvtColor(img_train, cv2.COLOR_BGR2GRAY)

## SIFT 검출기 생성
sift = cv2.xfeatures2d.SIFT_create()

## SIFT 기반 keypoint 검출 및 descriptor 생성
kp_query, des_query = sift.detectAndCompute(image = gray_query,
                                  mask = None)
kp_train, des_train = sift.detectAndCompute(gray_train, None)

## BFMatcher 객체 생성
bf = cv2.BFMatcher(normType = cv2.NORM_L2,
                   crossCheck = False)

## descriptor 간 매칭 수행
matches = bf.match(queryDescriptors = des_query,
                   trainDescriptors = des_train)

## distance를 기준으로 오름차순 정렬
matches = sorted(matches, key = lambda x:x.distance)

## 첫 10개의 매칭만 그리기
## https://docs.opencv.org/2.4/modules/features2d/doc/drawing_function_of_keypoints_and_matches.html#drawmatches
res = cv2.drawMatches(img1 = img_query,
                      keypoints1 = kp_query,
                      img2 = img_train,
                      keypoints2 = kp_train,
                      matches1to2 = matches[:10],
                      outImg = None,
                      flags = 2)

cv2.imshow('result', res)
cv2.waitKey(0)

cv2.destroyAllWindows()